# Binary Ring-LWE

Defining the RING-LWE parameters and Quotient Ring $\frac{Z_{q}[x]}{x^n + 1}$

In [1]:
n = 256
q = 256

R = PolynomialRing(Integers(q),"y")
x = R.gen()
S = R.quotient(x^n + 1, 'x')
w = S.gen()

Code to generate small polynomials $p = \sum^{n - 1}_{i=0} p_i.x^{i}$ where $p_i\in[-r, r]$

In [2]:
def smallPolynomial(times):
    ans = 0 
    for i in range(n):
        bit = randint(0, 1)
        ans += w^i*bit
    
    return ans

#print(smallPolynomial(8))

### First example on Ring-LWE 

In [ ]:
for p in range(49, 61):
    percentage = float(p)/100
    #print(percentage)
    counter = 0 
    for exp in range(10):

        
        a = S.random_element()
        s = smallPolynomial(r)
        e = smallPolynomial(r)

        b = a*s + e

        #print("b:", b)
        #print("-----------------------------------------------------------------------------------------------------------------")
        #print("a:", a)
        #print("-----------------------------------------------------------------------------------------------------------------")
        #print("s:", s)
        #print("e:", e)
        coefficientsS = {}
        coefficientsE = {}

        while len(coefficientsS) + len(coefficientsE) < int(percentage*(2*n)):
            rand = randint(0, 2*n - 1)
            
            if rand < n: 
                coefficientsS[rand] = True
                
            else: 
                rand = rand % n
                
                coefficientsE[rand] = True
                
        #print(len(coefficientsS), " - ", len(coefficientsE), "=" ,len(coefficientsS) + len(coefficientsE))
        
        MatrixA = []
        Y = []
        for i in range(n):
            aux = a*w^(n - i - 1)
            if i in coefficientsE:      
                vetorA = [aux[n - j - 1] for j in range(n)]
                MatrixA.append(list(vetorA))   
                
                Y.append(b[i] - e[i])

        MMatrixA = []

        for (i, row) in enumerate(MatrixA):  
            newRow = []
            _sum = 0
            for j in range(n):
                if j in coefficientsS: 
                    _sum += s[j]*row[j]
                else: 
                    newRow.append(row[j])
            MMatrixA.append(list(newRow))
            Y[i] -= _sum
            
        # Guassian Elimination's method

        MA = matrix(GF(q), MMatrixA[::-1])

        Z = MA.solve_right(vector(GF(q), Y[::-1]))
        
        newS = 0
        idx = 0
        for i in range(n):
            if i in coefficientsS:
                newS += s[i]*w^i
                
            else:
                newS +=(Integer(Z[idx]))*w^i                
                
                idx += 1
                
        if newS == s: 
            counter += 1 
            
        #print(exp, " -> ", len(MMatrixA), "equations and ",len(MMatrixA[0]), "variables  ==>", newS == s)    
        
    print(percentage, "Recovered keys:", counter)     
        